<a href="https://colab.research.google.com/github/jhk990602/datapractice/blob/main/RandomForest%2CExtraTree%2CGradientBoosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#정형데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘이 앙상블 학습, 앙상블 학습을 적용할 때 가장 먼저 랜덤 포레스트 시도 추천?
#부트스트랩방식> 가방에 1000개의 샘플이 있을 때 먼저 1개를 뽑고, 다시 가방에 넣어 그 다음 샘플을 뽑는 방식
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine-date')
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

In [7]:
#cross_validate()함수를 사용해 교차검증, RandomForestClassifier는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정하여 CPU사용하는 것이 좋음
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs= -1) #retrun_train_score 매개변수를 True로 지정하면 검증점수 뿐만 아니라 훈련세트에 대한 점수도 반환
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [8]:
#랜덤포레스트는 결정트리의 앙상블이기에 DecisionTreeClassifier가 제공하는 중요 매개변수 모두제공, 랜덤포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한것
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [10]:
#RandomForestClassifier에는 자체적으로 모델을 평가하는 점수를 얻을 수 있다. 부트스트랩 샘플에 포함되지 않고 남는 샘플을 사용하여 부트스트램 샘플로 훈련한 결증 트리를 평가(검증세트역할)
rf = RandomForestClassifier(oob_score = True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)  #oob_score매개변수를 True로 지정하면, 각 결정트리의 OOB 점수를 평균하여 출력

0.8934000384837406


랜덤포레스트와 엑스트라트리의 차이점은 부트스트랩샘플을 사용하지 않는다는점, 결정트리를 만들때 전체 훈련 세트를 사용, 대신 노드를 분할할때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할

In [11]:
#엑스트리트리는 하나의 결정트리에서 특성을 무작위로 분할한다면 성능이 낮아지겠지만, 많은트리를 앙상블하기때문에 과대적합을 막고 검증세트의 점수를 높이는 효과가 있음
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et,train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [12]:
#엑스트라 트리도 랜덤포레스트와 마찬가지로 특성 중요도를 제공,
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [15]:
#그레이디언트 부스팅은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블하는 방법, 기본적으로 깊이가 3인 결정트리를 100개 사용
#깊이가 얕은 결정트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능기대
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score =True, n_jobs = -1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


거의 과대적합이 되지 않았음, 그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강함

In [16]:
#학습률을 증가시키고, 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있음
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) #학습률(learning_rate의 기본값은 0.1)
scores = cross_validate(gb,train_input, train_target,return_train_score = True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 500개로 5배나 늘렸지만 과대적합을 잘 억제

In [17]:
#그레이디언트 부스팅도 특성 중요도를 제공
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [18]:
#히스토그램 기반 그레이디언트 부스팅은 정형 데이터를 다루는 머신러닝 알고리즘 중에 갖아 인기가 높은 알고리즘,
#이는 먼저 입력특성을 256개의 구간으로 나눔, 따라서 노드를 분할할때 최적의 분할을 매우 빠르게 찾을 수 있음.
#256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용, 따라서 누락된 특성이 있떠라도 이를 따로 전처리할 필요가 없음
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb,train_input, train_target,return_train_score =True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

/usr/local/lib/python3.10/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [19]:
#히스토그램 기반 그레이디언트 부스팅의 특성 중요도를 계산하기 위해 permutation_importance() 함수를 사용 > 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지 관찰하여 어느특성이 중요한지
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1) #n_repeat는 랜덤하게 섞을 횟수지정
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance()함수가 반환하는 객체는 반복하여 얻은 특성 중요도,평균,표준편차를 담고있음

In [21]:
result = permutation_importance(hgb,test_input,test_target,n_repeats=10, random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [22]:
#테스트 세트에서의 성능확인
hgb.score(test_input,test_target)

0.8723076923076923

In [25]:
#사이킷런라이브러리 말고도, XGBoost로도 히스토그램기반그레이디언트 부스팅 알고리즘을 구현가능
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist',random_state=42)
scores= cross_validate(xgb,train_input, train_target,return_train_score=True)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [27]:
#널리 사용하는 또 다른 히스토그램 기반 그레이디언트 부스팅 라이브러리는 LightGBM
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state = 42)
scores = cross_validate(lgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
